In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 128

dtype = None
load_in_4bit = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceH4/zephyr-7b-beta",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token = hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

HuggingFaceH4/zephyr-7b-beta does not have a padding token! Will use pad_token = <unk>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32, .
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=123,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### 데이터 준비
- 오직 완성된 텍스트만을 학습하고자 한다면,  [여기](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)


In [ ]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

# 훈련 데이터
dataset = load_dataset("Dongwookss/q_a_korean_futsal", split="train")

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Generating train split:   0%|          | 0/2332 [00:00<?, ? examples/s]

Map:   0%|          | 0/2332 [00:00<?, ? examples/s]

### 모델 훈련하기
TRL의 `SFTTrainer` 사용

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=11,
        gradient_accumulation_steps=2,  # 그래디언트 누적 단계
        warmup_steps=5,
        num_train_epochs=3 ,
        #max_steps = 106, # num_train_epochs와 max_steps중 하나만 설정해도되며 함께 선언 시 max_steps가 우선순위를 갖는다
        logging_steps = 10,
        learning_rate=2e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=123,
        output_dir="outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/2332 [00:00<?, ? examples/s]

- GPU의 현재 메모리 상태를 확인합니다.
- `torch.cuda.get_device_properties(0)`를 사용하여 첫 번째 GPU의 속성을 조회합니다.
- `torch.cuda.max_memory_reserved()`를 통해 현재 예약된 최대 메모리를 GB 단위로 계산합니다.
- GPU의 총 메모리 크기를 GB 단위로 계산합니다.
- GPU 이름과 최대 메모리 크기, 현재 예약된 메모리 크기를 출력합니다.


In [ ]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
14.396 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,332 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 11 | Gradient Accumulation steps = 2
\        /    Total batch size = 22 | Total steps = 318
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.042300
20,1.607800
30,1.477000
40,1.403600
50,1.346700
60,1.319900
70,1.298000
80,1.275100
90,1.245900
100,1.248700


In [ ]:
trainer_stats

TrainOutput(global_step=318, training_loss=1.2052770950509317, metrics={'train_runtime': 1259.166, 'train_samples_per_second': 5.556, 'train_steps_per_second': 0.253, 'total_flos': 3.843042166230221e+16, 'train_loss': 1.2052770950509317, 'epoch': 3.0})

In [ ]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1259.166 seconds used for training.
20.99 minutes used for training.
Peak reserved memory = 15.547 GB.
Peak reserved memory for training = 1.151 GB.
Peak reserved memory % of max memory = 70.133 %.
Peak reserved memory for training % of max memory = 5.192 %.


### 추론



In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID 초기화

    def __call__(self, input_ids, scores, **kwargs):
        return (self.stop_token_id in input_ids[0])


# end_token을 설정
stop_token = "<|end_of_text|>"
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[0]  # end_token의 ID를 인코딩

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList([StopOnToken(stop_token_id)])  # 정지 조건을 설정합니다.

(예시 1)

In [ ]:
from transformers import TextStreamer
# FastLanguageModel을 이용하여 빠른 추론속도 가속
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt.format(
            "",  # 지시사항
            "",  # 출력 - 공백
        )
    ],
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=512,  # 최대 생성 토큰 수를 설정
    stopping_criteria=stopping_criteria
)

(예시2)

(예시 3)

# 모델 저장

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# model.save_pretrained("경로")  # 모델을 로컬에 저장
model.push_to_hub("Dongwookss/small_fut_final", token = hf_token) # 모델을 온라인 허브에 저장

만약 우리가 저장한 LoRA 어댑터를 추론을 위해 불러오고 싶다면, `False`를 `True`로 설정하세요.


### VLLM을 위한 float16 저장


저장

In [ ]:
base_model = "HuggingFaceH4/zephyr-7b-beta"  # 병합을 수행할 베이스 모델
huggingface_token = hf_token # HuggingFace 토큰
huggingface_repo = "업로드 할 repo" # 모델을 업로드할 repository
save_method = "merged_16bit"              # "merged_4bit", "merged_4bit_forced", "merged_16bit", "lora"


### 옵션 1) 로컬에 저장

In [ ]:
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.74 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:08<00:00,  3.65it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


### 옵션 2) HuggingFace 에 업로드

In [ ]:
# Hub 에 업로드
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.92 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:54<00:00,  3.59s/it]


Unsloth: Saving to organization with address Dongwookss/unsloth_l3_futfut
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address Dongwookss/unsloth_l3_futfut
Unsloth: Saving Dongwookss/unsloth_l3_futfut/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Dongwookss/unsloth_l3_futfut/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Dongwookss/unsloth_l3_futfut/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Dongwookss/unsloth_l3_futfut/pytorch_model-00004-of-00004.bin...
Unsloth: Uploading all files... Please wait...


pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/unsloth_l3_futfut


### GGUF 변환


### 옵션1) 로컬 저장

In [ ]:
# Quantization 방식 설정
quantization_method = "q8_0" # "f16" "q8_0" "q4_k_m" "q5_k_m"

In [ ]:
model.save_pretrained_gguf(
    "./content/drive/MyDrive",
    tokenizer=tokenizer,
    quantization_method=quantization_method,
)

### 옵션2) HuggingFace 허브에 업로드

In [ ]:
# Quantization 방식 설정
quantization_method = "q8_0" # "f16" "q8_0" "q4_k_m" "q5_k_m"

In [ ]:
# Hub 에 GGUF 업로드
model.push_to_hub_gguf(
    "경로",
    tokenizer,
    quantization_method=quantization_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 35.54 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:30<00:00,  1.05it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at Dongwookss/zephyr_tuning-gguf into bf16 GGUF format.
The output location will be ./Dongwookss/zephyr_tuning-gguf-unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: zephyr_tuning-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 32768
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: 

zephyr_tuning-gguf-unsloth.Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/Dongwookss/zephyr_tuning-gguf
